In [1]:
### import libs
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

import gensim
import numpy as np
import pandas as pd
import math
import spacy

from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

import os, re, operator, warnings
warnings.filterwarnings('ignore')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eric\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('../tweets_01-08-2021.csv')[:1000]

In [3]:
df_result = df[['text', 'date']]

In [4]:
df_result['text_lower'] = df_result['text'].str.lower()
df_result

,text,date,text_lower
0,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,republicans and democrats have both created ou...
1,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,i was thrilled to be back in the great city of...
2,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,rt @cbs_herridge: read: letter to surveillance...
3,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,the unsolicited mail in ballot scam is a major...
4,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,rt @mzhemingway: very friendly telling of even...
...,...,...,...
995,But 2020 is a long way from over! https://t.co...,2020-11-25 15:44:26,but 2020 is a long way from over! https://t.co...
996,RT @ScottAdamsSays: The most corrupt instituti...,2020-11-14 05:29:30,rt @scottadamssays: the most corrupt instituti...
997,RT @jacobkschneider: Boston Herald endorses Pr...,2020-10-27 15:29:31,rt @jacobkschneider: boston herald endorses pr...
998,RT @marklevinshow: The Boston Herald endorses ...,2020-10-27 15:28:54,rt @marklevinshow: the boston herald endorses ...


In [5]:
df_result['text_no_num'] = df_result['text_lower'].map(lambda s: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s))
df_result

,text,date,text_lower,text_no_num
0,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,republicans and democrats have both created ou...,republicans and democrats have both created ou...
1,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,i was thrilled to be back in the great city of...,i was thrilled to be back in the great city of...
2,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,rt @cbs_herridge: read: letter to surveillance...,rt @cbs_herridge: read: letter to surveillance...
3,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,the unsolicited mail in ballot scam is a major...,the unsolicited mail in ballot scam is a major...
4,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,rt @mzhemingway: very friendly telling of even...,rt @mzhemingway: very friendly telling of even...
...,...,...,...,...
995,But 2020 is a long way from over! https://t.co...,2020-11-25 15:44:26,but 2020 is a long way from over! https://t.co...,but is a long way from over! https://t.co/usdk...
996,RT @ScottAdamsSays: The most corrupt instituti...,2020-11-14 05:29:30,rt @scottadamssays: the most corrupt instituti...,rt @scottadamssays: the most corrupt instituti...
997,RT @jacobkschneider: Boston Herald endorses Pr...,2020-10-27 15:29:31,rt @jacobkschneider: boston herald endorses pr...,rt @jacobkschneider: boston herald endorses pr...
998,RT @marklevinshow: The Boston Herald endorses ...,2020-10-27 15:28:54,rt @marklevinshow: the boston herald endorses ...,rt @marklevinshow: the boston herald endorses ...


In [6]:
import string

df_result['text_no_punct'] = df_result['text_no_num'].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
df_result

,text,date,text_lower,text_no_num,text_no_punct
0,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,republicans and democrats have both created ou...,republicans and democrats have both created ou...,republicans and democrats have both created ou...
1,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,i was thrilled to be back in the great city of...,i was thrilled to be back in the great city of...,i was thrilled to be back in the great city of...
2,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,rt @cbs_herridge: read: letter to surveillance...,rt @cbs_herridge: read: letter to surveillance...,rt cbsherridge read letter to surveillance cou...
3,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,the unsolicited mail in ballot scam is a major...,the unsolicited mail in ballot scam is a major...,the unsolicited mail in ballot scam is a major...
4,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,rt @mzhemingway: very friendly telling of even...,rt @mzhemingway: very friendly telling of even...,rt mzhemingway very friendly telling of events...
...,...,...,...,...,...
995,But 2020 is a long way from over! https://t.co...,2020-11-25 15:44:26,but 2020 is a long way from over! https://t.co...,but is a long way from over! https://t.co/usdk...,but is a long way from over httpstcousdkmvgmhk
996,RT @ScottAdamsSays: The most corrupt instituti...,2020-11-14 05:29:30,rt @scottadamssays: the most corrupt instituti...,rt @scottadamssays: the most corrupt instituti...,rt scottadamssays the most corrupt institution...
997,RT @jacobkschneider: Boston Herald endorses Pr...,2020-10-27 15:29:31,rt @jacobkschneider: boston herald endorses pr...,rt @jacobkschneider: boston herald endorses pr...,rt jacobkschneider boston herald endorses pres...
998,RT @marklevinshow: The Boston Herald endorses ...,2020-10-27 15:28:54,rt @marklevinshow: the boston herald endorses ...,rt @marklevinshow: the boston herald endorses ...,rt marklevinshow the boston herald endorses tr...


In [7]:
stop_words = stopwords.words('english')
df_result['text_no_stopwords'] = df_result['text_no_punct'].apply(lambda s: ' '.join([word for word in s.split() if word not in (stop_words)]))
df_result

,text,date,text_lower,text_no_num,text_no_punct,text_no_stopwords
0,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,republicans and democrats have both created ou...,republicans and democrats have both created ou...,republicans and democrats have both created ou...,republicans democrats created economic problems
1,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,i was thrilled to be back in the great city of...,i was thrilled to be back in the great city of...,i was thrilled to be back in the great city of...,thrilled back great city charlotte north carol...
2,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,rt @cbs_herridge: read: letter to surveillance...,rt @cbs_herridge: read: letter to surveillance...,rt cbsherridge read letter to surveillance cou...,rt cbsherridge read letter surveillance court ...
3,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,the unsolicited mail in ballot scam is a major...,the unsolicited mail in ballot scam is a major...,the unsolicited mail in ballot scam is a major...,unsolicited mail ballot scam major threat demo...
4,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,rt @mzhemingway: very friendly telling of even...,rt @mzhemingway: very friendly telling of even...,rt mzhemingway very friendly telling of events...,rt mzhemingway friendly telling events comeys ...
...,...,...,...,...,...,...
995,But 2020 is a long way from over! https://t.co...,2020-11-25 15:44:26,but 2020 is a long way from over! https://t.co...,but is a long way from over! https://t.co/usdk...,but is a long way from over httpstcousdkmvgmhk,long way httpstcousdkmvgmhk
996,RT @ScottAdamsSays: The most corrupt instituti...,2020-11-14 05:29:30,rt @scottadamssays: the most corrupt instituti...,rt @scottadamssays: the most corrupt instituti...,rt scottadamssays the most corrupt institution...,rt scottadamssays corrupt institutions individ...
997,RT @jacobkschneider: Boston Herald endorses Pr...,2020-10-27 15:29:31,rt @jacobkschneider: boston herald endorses pr...,rt @jacobkschneider: boston herald endorses pr...,rt jacobkschneider boston herald endorses pres...,rt jacobkschneider boston herald endorses pres...
998,RT @marklevinshow: The Boston Herald endorses ...,2020-10-27 15:28:54,rt @marklevinshow: the boston herald endorses ...,rt @marklevinshow: the boston herald endorses ...,rt marklevinshow the boston herald endorses tr...,rt marklevinshow boston herald endorses trump ...
